In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
from datetime import datetime
import threading
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import date, timedelta
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup

In [13]:
driver.quit()

In [120]:
chrome_driver_path = 'chromedriver.exe'
chrome_options = Options()
service = Service(executable_path=chrome_driver_path)
driver = webdriver.Chrome()

In [121]:
all_news = []
# Задаем начальную дату
start_date = date(2022, 2, 25)
# И конечную дату
end_date = date(2025, 3, 25)
current_date = start_date



try:
  while current_date <= end_date:

    # Формируем URL
    date_str = current_date.strftime("%Y-%m-%d")
    url = f"https://www.kommersant.ru/archive/theme/3423/day/{date_str}"
    # Загружаем страницу
    driver.get(url)
    while True:
        try:
            button_locators = (By.CLASS_NAME, "js-archive-more-button") #  замените на нужный вам класс
            buttons = driver.find_elements(*button_locators) 
            flag = False
            for btn in buttons:
                if btn.text == ' Показать еще ':
                    driver.execute_script("arguments[0].scrollIntoView();", btn)
                    btn[0].click()
                    time.sleep(2)
                    ok = True
            if not flag:
                break
        except Exception as x:
            print("Кнопка обрабатывается. Подождите...")
    print(f"Processing news for {current_date}")
    try:
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # Получаем все новости
        news_list = soup.find_all('article')
        if not news_list:
            print(f"No news for {current_date}")
            continue

        for news in news_list:
            try:
                # Извлечение ссылки из атрибута data-article-url
                news_url = news.get('data-article-url')
                if not news_url:
                     news_url = "Ссылка не найдена"
                # Извлечение заголовка из атрибута data-article-title
                news_title = news.get('data-article-title')
                if not news_title:
                     news_title = "Заголовок не найден"
                # Извлечение даты из тега
                news_description = news.get('data-article-description')
                if not news_description:
                     news_description = "Описание не найдено не найден"
                date_element = news.find('p', class_="uho__tag")
                if date_element:
                   news_date = date_element.text.strip()
                else:
                  news_date = "Дата не найдена"
        
                # Извлечение тегов
                tag_elements = news.find_all('a', class_="tag_list__link")
                tags = [tag.text.strip() for tag in tag_elements]
        
                article_data = {
                 "url": news_url,
                 "description": news_description,
                 "title": news_title,
                 "date": news_date,
                  "tags": tags
                }
                all_news.append(article_data)
            except Exception as e:
                print(f"An error occurred while processing one news: {e}")
            
    except TimeoutException:
            print(f"Timeout while loading page for {current_date}")
    except Exception as e:
            print(f"An unexpected error occurred: {e}")

    current_date += timedelta(days=1)

except Exception as e:
     print(f"An error occurred: {e}")
finally:
    driver.quit()

Processing news for 2023-08-26
Processing news for 2023-08-27
Processing news for 2023-08-28
Processing news for 2023-08-29
Processing news for 2023-08-30
Processing news for 2023-08-31
Processing news for 2023-09-01
Processing news for 2023-09-02
Processing news for 2023-09-03
Processing news for 2023-09-04
Processing news for 2023-09-05
Processing news for 2023-09-06
Processing news for 2023-09-07
Processing news for 2023-09-08
Processing news for 2023-09-09
Processing news for 2023-09-10
Processing news for 2023-09-11
Processing news for 2023-09-12
Processing news for 2023-09-13
Processing news for 2023-09-14
Processing news for 2023-09-15
Processing news for 2023-09-16
Processing news for 2023-09-17
Processing news for 2023-09-18
Processing news for 2023-09-19
Processing news for 2023-09-20
Processing news for 2023-09-21
Processing news for 2023-09-22
Processing news for 2023-09-23
Processing news for 2023-09-24
Processing news for 2023-09-25
Processing news for 2023-09-26
Processi

In [122]:
filename = 'news_svo_v3.xlsx'
df = pd.DataFrame(all_news)
df.to_excel(filename, index=False)  # index=False убирает столбец с индексами
print(f"Data saved to {filename}")

Data saved to news_svo_v3.xlsx


In [108]:
len(all_news)

19182